# HapCUT2 phasing for samples
  
**Date:** 2023-Oct-06  
**last update:** 2023-Oct-20

Molecular phasing is performed individually on all samples with hapCut2, generally following the steps below -

1. generate two temporary VCF files - one with only heterozygous sites, and the other one with all sites
2. HAPCUT2 pipeline, with basically three steps:
    - Parse the bam file for reads that correspond to the heterozygous sites for HAPCUT2 using the extractHAIRs utility with the 10X flag on to generate a BX-tagged, unlinked fragment file
    - Link the fragments using HAPCUT2's LinkFragments.py utility.
    - Run HAPCUT2 proper, with VCF output option
3. Perform some basic data extraction from the HAPCUT2 output
4. Merge the full VCF file with the HAPCUT2-derived VCF files and annotating the INFO and FORMAT fields accordingly
5. Clean-up

In [ ]:
module load htslib/1.18 bcftools/1.18 samtools/1.18
unset $PYTHONPATH
export PYTHONPATH=$PYTHONPATH:~/.local/lib/python3.9/site-packages
export PATH=$PATH:~/_softwares/HapCUT2/build
export PATH=$PATH:~/_softwares/HapCUT2/utilities

module list

[HapCUT2](https://github.com/vibansal/HapCUT2/tree/master#readme) is for diploid organisms only and can assemble haplotypes for one individual at a time. VCF input should contain variants and genotypes for a single diploid individual.

In [2]:
cd /nfs/scistore18/bartogrp/apal/snap_hap/variants/molphase
grep -w 'High' ~/snap_hap/sample_info/samples_Amajus_SnapHap_LastUpdate-2023-10.txt > samples_gt5x.list
grep -w 'Low' ~/snap_hap/sample_info/samples_Amajus_SnapHap_LastUpdate-2023-10.txt > samples_lt5x.list

## Molecular phasing 1 sample

In [5]:
grep -w "10xNEW-Plate10-80_Am_Pla_pb3432_v3.5" ~/snap_hap/variants/molphase/samples_gt5x.list | head -1

PB3432	pb3432	2x-N710-76_Am_Pla_pb3432	10xNEW-Plate10-80_Am_Pla_pb3432_v3.5	10xNEW	10xNEW_PB3432	PB3432_10xNEW	Am	Pla	5.1464	34.7223	35.73	40.9774	183	895	791	High	42.32448359	2.064860881	1216.624	YF1	NA	/nfs/scistore18/bartogrp/apal/snap_hap/bams/v3.5/bams_merged_2023/10xNEW-Plate10-75_Am_Pla_pb3400_v3.5.sorted.BXnum.merged.bam


### Initiate variables

In [6]:
chrom=Chr6
baseDIR=/nfs/scistore18/bartogrp/apal/snap_hap/variants/molphase/${chrom}/indv-molphase
sampleList=~/snap_hap/variants/molphase/samples_gt5x.list
bamList=~/snap_hap/sample_info/bam_info/bams_Am_all.txt
vcf=~/snap_hap/variants/stitch/${chrom}/Am_all_stitch_${chrom}_SnpGap5_biSNPs_filtered-DP_500-7732_QUAL20_MQ30.PL.vcf.gz
threshold=10

# sample=$(tail +2 $sampleList | cut -f4 | sed -n "${SLURM_ARRAY_TASK_ID}p")
sample=10xNEW-Plate10-80_Am_Pla_pb3432_v3.5
bam=$(cat $bamList | grep $sample)

# coverage=$(tail +2 $sampleList | cut -f2 | sed -n "${SLURM_ARRAY_TASK_ID}p")
if [ ! -d $baseDIR/${chrom}_${sample} ]; then mkdir -p $baseDIR/${chrom}_${sample}; fi
sampleDIR=$baseDIR/${chrom}_${sample}

sampleHetVCF=${chrom}_${sample}
sampleHomVCF=${chrom}_${sample}

unLinkedFragments=${chrom}_${sample}
LinkedFragments=${chrom}_${sample}

outPhased=${chrom}_${sample}_thres${threshold}.hapcut2.output

cd $sampleDIR
echo sampleDIR: $sampleDIR
echo chrom: $chrom
echo sample: $sample
# echo coverage: $coverage
echo BAM: $bam
echo VCF: $vcf
echo sample HET VCF: $sampleHetVCF
echo hapcut2 output: $outPhased

sampleDIR: /nfs/scistore18/bartogrp/apal/snap_hap/variants/molphase/Chr6/indv-molphase/Chr6_10xNEW-Plate10-80_Am_Pla_pb3432_v3.5
chrom: Chr6
sample: 10xNEW-Plate10-80_Am_Pla_pb3432_v3.5
BAM: /nfs/scistore18/bartogrp/apal/snap_hap/bams/v3.5/bams_merged_2023/10xNEW-Plate10-80_Am_Pla_pb3432_v3.5.sorted.BXnum.merged.bam
VCF: /nfs/scistore18/bartogrp/apal/snap_hap/variants/stitch/Chr6/Am_all_stitch_Chr6_SnpGap5_biSNPs_filtered-DP_500-7732_QUAL20_MQ30.PL.vcf.gz
sample HET VCF: Chr6_10xNEW-Plate10-80_Am_Pla_pb3432_v3.5
hapcut2 output: Chr6_10xNEW-Plate10-80_Am_Pla_pb3432_v3.5_thres10.hapcut2.output


### Preprocessing 

In [10]:
# time bcftools view -s $sample -r $chrom $vcf | awk '/^#/;/Chr/ {OFS="\t"}; !/^#/ && $10~/^0\/1/' > ${sampleHetVCF}.het.vcf
time bcftools view -s $sample -r Chr6:1000000-1100000 $vcf | awk '/^#/;/Chr/ {OFS="\t"}; !/^#/ && $10~/^0\/1/' > ${sampleHetVCF}.het.vcf
echo -e 'No of variants in Chr6:1000000-1100000:' $(bcftools view -H ${sampleHetVCF}.het.vcf | wc -l)


real	0m2.929s
user	0m2.880s
sys	0m0.053s
No of variants in Chr6:1000000-1100000: 1008


### HapCUT2

In [13]:
# Extract unlinked fragments
time extractHAIRS --10X 1 --bam $bam --VCF ${sampleHetVCF}.het.vcf --region $chrom --out ${unLinkedFragments}.unlinked.fragments
echo -e "\nNo. of unlinked fragments:" $(wc -l ${unLinkedFragments}.unlinked.fragments)'\n'

# Convert unlinked to linked fragments
time python3 ~/_softwares/HapCUT2/utilities/LinkFragments.py  --bam $bam --VCF ${sampleHetVCF}.het.vcf --fragments ${unLinkedFragments}.unlinked.fragments --out ${LinkedFragments}.linked.fragments -d 50000
echo -e "\nNo. of linked fragments" $(wc -l ${LinkedFragments}.linked.fragments)'\n'

# Phase with HapCUT2
time HAPCUT2 --fragments ${LinkedFragments}.linked.fragments --VCF ${sampleHetVCF}.het.vcf --out ${outPhased} --nf 1 --threshold $threshold --error_analysis_mode 1 --call_homozygous 1 --outvcf 1  --v 0
echo -e "\nNo. of total variants" $(bcftools view -H $outPhased.phased.VCF | wc -l)
echo -e "\nNo. of phased variants" $(bcftools view -H $outPhased.phased.VCF | grep -vc "0/1")
echo -e "\nNo. of unphased variants" $(bcftools view -H $outPhased.phased.VCF | grep -c "0/1")


Extracting haplotype informative reads from bamfiles /nfs/scistore18/bartogrp/apal/snap_hap/bams/v3.5/bams_merged_2023/10xNEW-Plate10-80_Am_Pla_pb3432_v3.5.sorted.BXnum.merged.bam minQV 13 minMQ 20 maxIS 1000 

VCF file Chr6_10xNEW-Plate10-80_Am_Pla_pb3432_v3.5.het.vcf has 1008 variants 
adding chrom Chr6 to index 
vcffile Chr6_10xNEW-Plate10-80_Am_Pla_pb3432_v3.5.het.vcf chromosomes 1 hetvariants 1008 variants 1008 
detected 0 variants with two non-reference alleles, these variants will not be phased
reading sorted bamfile /nfs/scistore18/bartogrp/apal/snap_hap/bams/v3.5/bams_merged_2023/10xNEW-Plate10-80_Am_Pla_pb3432_v3.5.sorted.BXnum.merged.bam 
processing reads mapped to chrom "Chr6" 
processed 2000000 reads
final cleanup of fragment list: 1392 current chrom 0 prev 0 

real	0m4.264s
user	0m4.152s
sys	0m0.081s

No. of unlinked fragments: 896 Chr6_10xNEW-Plate10-80_Am_Pla_pb3432_v3.5.unlinked.fragments

Linking 10X fragments on chromosome: Chr6
  reading bedfile...
  generating new

In [18]:
head -50 $outPhased

BLOCK: offset: 1 len: 2 phased: 2 SPAN: 9 fragments 1
1	0	1	Chr6	1000020	G	A	0/1:./.:0.441,0.547,0.012:0.571:1,0,17	0	100.00	29.99	1
2	1	0	Chr6	1000029	G	T	0/1:./.:0.127,0.847,0.026:0.899:8,0,15	0	29.99	29.99	1
******** 
BLOCK: offset: 4 len: 1005 phased: 933 SPAN: 98032 fragments 364
4	0	1	Chr6	1000090	T	C	0/1:./.:0.269,0.724,0.006:0.737:4,0,21	0	100.00	36.99	1
5	0	1	Chr6	1000092	C	T	0/1:./.:0.253,0.72,0.027:0.775:5,0,14	0	37.00	36.99	1
6	0	1	Chr6	1000107	C	T	0/1:./.:0.214,0.775,0.011:0.797:6,0,18	0	74.00	36.99	1
7	0	1	Chr6	1000116	C	T	0/1:./.:0.244,0.741,0.015:0.771:5,0,17	0	100.00	36.99	1
8	1	0	Chr6	1000176	C	T	0/1:./.:0.184,0.56,0.256:1.072:5,0,3	0	100.00	36.99	1
9	0	1	Chr6	1000205	A	G	0/1:./.:0.388,0.501,0.11:0.722:1,0,7	0	100.00	100.00	2
10	0	1	Chr6	1000259	G	A	0/1:./.:0.223,0.522,0.254:1.031:4,0,3	0	100.00	100.00	2
11	0	1	Chr6	1000262	G	A	0/1:./.:0.458,0.541,0.001:0.543:1,0,27	0	100.00	3.01	2
12	1	0	Chr6	1000266	G	A	0/1:./.:0.37,0.625,0.004:0.634:2,0,22	0	37.00	25.00	3
13	1	0	Ch

In [22]:
bcftools view -h $outPhased.phased.VCF | cut -f1-2,4-5,8-10 |  tail -1
bcftools view -H $outPhased.phased.VCF | head -15 | cut -f1-2,4-5,8-10

#CHROM	POS	REF	ALT	INFO	FORMAT	10xNEW-Plate10-80_Am_Pla_pb3432_v3.5
[main_vcfview] Error: cannot write to (null)
Chr6	1000020	G	A	EAF=0.45818;INFO_SCORE=0.49317;HWE=5.37e-23;ERC=985.466;EAC=1082.5;PAF=0.47654;AC=1;AN=2	GT:PQ:PD:PS	1|0:30:1:1000020
Chr6	1000029	G	T	EAF=0.04558;INFO_SCORE=0.3758;HWE=1;ERC=83.7223;EAC=1920.45;PAF=0.04177;AC=1;AN=2	GT:PQ:PD:PS	0|1:30:1:1000020
Chr6	1000077	C	T	EAF=0.04814;INFO_SCORE=0.28625;HWE=1;ERC=39.2469;EAC=1506.91;PAF=0.02538;AC=1;AN=2	GT:PG:GP:DS:PL:PS	0/1:./.:0.238,0.754,0.008:0.77:5,0,20:.
Chr6	1000090	T	C	EAF=0.04413;INFO_SCORE=0.28503;HWE=1;ERC=40.2703;EAC=1515.87;PAF=0.02588;AC=1;AN=2	GT:PQ:PD:PS	1|0:37:1:1000090
Chr6	1000092	C	T	EAF=0.13937;INFO_SCORE=0.28352;HWE=0.00298;ERC=123.963;EAC=1396.21;PAF=0.08155;AC=1;AN=2	GT:PQ:PD:PS	1|0:37:1:1000090
Chr6	1000107	C	T	EAF=0.0991;INFO_SCORE=0.35638;HWE=0.0827;ERC=85.0538;EAC=1438.16;PAF=0.05584;AC=1;AN=2	GT:PQ:PD:PS	1|0:37:1:1000090
Chr6	1000116	C	T	EAF=0.14335;INFO_SCORE=0.37598;HWE=0.00171;ERC=114.9

In [15]:
python3 ~/_softwares/HapCUT2/utilities/calculate_haplotype_statistics.py -v1 $outPhased.phased.VCF

ERROR: Missing required arguments.
--vcf1 and --vcf2 options are required


: 1

## Cluster implementation

### Chr6

In [2]:
cd /nfs/scistore18/bartogrp/apal/snap_hap/variants/jobs/molphase_allSamples
chrom=Chr6
stitchRun=stitchRun1
baseDIR=/nfs/scistore18/bartogrp/apal/snap_hap/variants/molphase/${chrom}/indv-molphase
bamList=~/snap_hap/sample_info/bam_info/bams_Am_all.txt
# vcf=~/snap_hap/variants/stitch/${chrom}/Am_all_stitch_${chrom}_SnpGap5_biSNPs_filtered-DP_500-7732_QUAL20_MQ30.PL.vcf.gz
vcf=~/snap_hap/variants/stitch/${chrom}/Am_all_${stitchRun}_${chrom}.final.vcf.gz
threshold=10
headerFile=~/snap_hap/variants/molphase/molphase.header

In [3]:
cat $headerFile

##INFO=<ID=HAPCUT,Number=1,Type=String,Description="HapCUT2 Phase">
##FORMAT=<ID=GX,Number=1,Type=String,Description="HapCUT2 Phased Genotype">
##FORMAT=<ID=PS,Number=1,Type=Integer,Description="ID of Phase Set for Variant">
##FORMAT=<ID=PQ,Number=1,Type=Integer,Description="Phred QV indicating probability that this variant is incorrectly phased relative to the haplotype">
##FORMAT=<ID=PD,Number=1,Type=Integer,Description="phased Read Depth">


#### Individual Molphase:  >5x coverage

In [ ]:
sampleList=~/snap_hap/variants/molphase/samples_gt5x.list
sbatch --array=1-180 -J molphase_Chr6_gt5x ~/snap_hap/_scripts/sbatch/phase/job-molphase_allSamples.sbatch $chrom $baseDIR $sampleList $bamList $vcf $threshold
# ##Postprocessing 
# ##NB: Ideally, both molphase and postprocesing would be run altogether.
# sbatch --array=1-180 -J postPhase_Chr6_gt5x ~/snap_hap/_scripts/sbatch/phase/job-molphase_allSamples.sbatch.sh $chrom $baseDIR $sampleList $bamList $vcf $threshold

```
#SBATCH --partition=defaultp
#SBATCH --time=48:00:00
#SBATCH --ntasks=1
#SBATCH --ntasks-per-node=1
#SBATCH --cpus-per-task=1
#SBATCH --mem-per-cpu=16G
```

#### Individual Molphase:  <5x coverage

In [ ]:
sampleList=~/snap_hap/variants/molphase/samples_lt5x.list
sbatch --array=1-894 -J molphase_Chr6_lt5x ~/snap_hap/_scripts/sbatch/phase/job-molphase_allSamples.sbatch $chrom $baseDIR $sampleList $bamList $vcf $threshold
# ##Postprocessing 
# ##NB: Ideally, both molphase and postprocesing would be run altogether.
# sbatch --array=1-894 -J postPhase_Chr6_lt5x ~/snap_hap/_scripts/sbatch/phase/job-molphase_allSamples.sbatch.sh $chrom $baseDIR $sampleList $bamList $vcf $threshold

```
#SBATCH --partition=defaultp
#SBATCH --time=03:00:00
#SBATCH --ntasks=1
#SBATCH --ntasks-per-node=1
#SBATCH --cpus-per-task=1
#SBATCH --mem-per-cpu=8G
```

### For all chromosomes

In [ ]:
for chrom in Chr{1..8}
do
    echo $chrom
    if [ ! -d ~/snap_hap/variants/jobs/molphase_allSamples/${chrom} ]; then mkdir -p ~/snap_hap/variants/jobs/molphase_allSamples/$chrom; fi

    stitchRun=stitchRun1
    baseDIR=/nfs/scistore18/bartogrp/apal/snap_hap/variants/molphase/${chrom}/indv-molphase
    bamList=~/snap_hap/sample_info/bam_info/bams_Am_all.txt
    vcf=~/snap_hap/variants/stitch/${chrom}/Am_all_${stitchRun}_${chrom}.final.vcf.gz
    threshold=10
    headerFile=~/snap_hap/variants/molphase/molphase.header

    cd /nfs/scistore18/bartogrp/apal/snap_hap/variants/jobs/molphase_allSamples/$chrom
    
    sampleList_highCov=~/snap_hap/variants/molphase/samples_gt5x.list
    sbatch --array=1-180 -J ${chrom}_gt5x ~/snap_hap/_scripts/sbatch/phase/job-molphase_allSamples.sbatch.sh $chrom $baseDIR \
            $sampleList_highCov $bamList $vcf $threshold

    sampleList_lowCov=~/snap_hap/variants/molphase/samples_lt5x.list
    sbatch --array=1-894 -J ${chrom}_lt5x ~/snap_hap/_scripts/sbatch/phase/job-molphase_allSamples.sbatch.sh $chrom $baseDIR \
            $sampleList_lowCov $bamList $vcf $threshold
done

-----

#### Merge all individual molphase VCFs

In [ ]:
## Merge VCFs
echo -e '\nMerging sample VCFs'
srun time bcftools merge --threads ${SLURM_CPUS_PER_TASK} -Oz -o $outVCF -l $sampleVcfList 
tabix -f $outVCF
## Create GT file
srun time bcftools query -f '%CHROM\t%POS[\t%GT]\n' $vcf > Am_all_molphased_Chr6.gt

In [ ]:
## Merge all combined sample VCF (cluster implementation)
cd /nfs/scistore18/bartogrp/apal/snap_hap/variants/jobs/molphase_allSamples/mergeVcfs_Chr6
chrom=Chr6
baseDIR=/nfs/scistore18/bartogrp/apal/snap_hap/variants/molphase/${chrom}
realpath ${baseDIR}/indv-molphase/*/*combined.vcf.gz > ${baseDIR}/molphaseVcfs.list
sampleVcfList=${baseDIR}/molphaseVcfs.list

sbatch -J postPhase-mergeVcfs_${chrom} ~/snap_hap/_scripts/sbatch/phase/job-molphase_mergeVcfs.sbatch $chrom $baseDIR $sampleVcfList

In [ ]:
### For all chromosomes (cluster implementation)
for chrom in Chr{1..8}
do
    echo $chrom
    
    if [ ! -d ~/snap_hap/variants/jobs/molphase_allSamples/${chrom}_mergeVCFs ]; 
    then 
        mkdir -p ~/snap_hap/variants/jobs/molphase_allSamples/${chrom}_mergeVCFs; 
    fi

    cd ~/snap_hap/variants/jobs/molphase_allSamples/${chrom}_mergeVCFs
    
    stitchRun=stitchRun1
    baseDIR=/nfs/scistore18/bartogrp/apal/snap_hap/variants/molphase/${chrom}
    realpath ${baseDIR}/indv-molphase/*/*combined.vcf.gz > ${baseDIR}/molphaseVcfs.list
    sampleVcfList=${baseDIR}/molphaseVcfs.list
    
    sbatch -J ${chrom}_merge ~/snap_hap/_scripts/sbatch/phase/job-molphase_mergeVcfs.sbatch.sh \
        $chrom $baseDIR $sampleVcfList
done

Merging takes around ~3-4 hrs.

----

In [ ]:
vcf=~/snap_hap/variants/molphase/Chr6/Am_all_stitchRun1_Chr6.molphased.vcf.gz
TotalSites=$(bcftools view -H $vcf | wc -l)
echo -e "Total no. of variant sites in Chr6:" $TotalSites

In [ ]:
### For all chromosomes
echo -e "Total no. of variant sites\n"
for chrom in Chr{1..8}
do
    vcf=~/snap_hap/variants/molphase/${chrom}/Am_all_stitchRun1_${chrom}.molphased.vcf.gz
    # TotalSites=$(bcftools view -H $vcf | wc -l)
    TotalSites=$(zgrep -c ^Chr $vcf)  
    echo -e $chrom':' $TotalSites
done

#### Get summary

In [ ]:
## Chr6
cd ~/snap_hap/variants/molphase/Chr6
paste <(head -1 ~/snap_hap/sample_info/samples_Amajus_SnapHap_LastUpdate-2023-10.txt) <(echo -e "sampleFolder\tunLinkedFragments\tLinkedFragments\tTotalHetSites\tPhasedSites\tunPhasedSites") > phaseVal_Chr6.txt

for sampleFolder in /nfs/scistore18/bartogrp/apal/snap_hap/variants/molphase/Chr6/indv-molphase/*;
do 
    
    sample=`basename $sampleFolder | cut -f3- -d_`
    grep -w $sample ~/snap_hap/sample_info/samples_Amajus_SnapHap_LastUpdate-2023-10.txt > tmpSample.txt
    
    unLinkedFragments=`cat $sampleFolder/*.unlinked.fragments | wc -l`
    LinkedFragments=`cat $sampleFolder/*.linked.fragments | wc -l`

    TotalHetSites=$(bcftools view -H $sampleFolder/*.het.vcf | wc -l)
    PhasedSites=$(bcftools view -H $sampleFolder/*.phased.VCF | grep -vc "0/1")
    unPhasedSites=$(bcftools view -H $sampleFolder/**.phased.VCF | grep -c "0/1")

    paste tmpSample.txt <(echo -e "$(basename $sampleFolder)\t$unLinkedFragments\t$LinkedFragments\t$TotalHetSites\t$PhasedSites\t$unPhasedSites") >> phaseVal_Chr6.txt
    echo `basename $sampleFolder` -> Done
    rm tmpSample.txt
done

In [7]:
### For all chromosomes

for chrom in Chr{1..8}
do
    echo $chrom
    baseDIR=~/snap_hap/variants/molphase/${chrom}
    cd $baseDIR

    paste <(head -1 ~/snap_hap/sample_info/samples_Amajus_SnapHap_LastUpdate-2023-10.txt) \
        <(echo -e "sampleFolder\tunLinkedFragments\tLinkedFragments\tTotalHetSites\tPhasedSites\tunPhasedSites") \
        > ./phaseVal_${chrom}.txt

    for sampleFolder in $baseDIR/indv-molphase/*;
    do
        sample=`basename $sampleFolder | cut -f3- -d_`
        grep -w $sample ~/snap_hap/sample_info/samples_Amajus_SnapHap_LastUpdate-2023-10.txt > tmpSample.txt

        unLinkedFragments=`cat $sampleFolder/*.unlinked.fragments | wc -l`
        LinkedFragments=`cat $sampleFolder/*.linked.fragments | wc -l`

        TotalHetSites=$(grep -c ^Chr $sampleFolder/*.het.vcf)
        PhasedSites=$(grep ^Chr  $sampleFolder/*.phased.VCF | grep -vc "0/1")
        unPhasedSites=$(grep ^Chr  $sampleFolder/*.phased.VCF | grep -c "0/1")

        paste tmpSample.txt <(echo -e "$(basename $sampleFolder)\t$unLinkedFragments\t$LinkedFragments\t$TotalHetSites\t$PhasedSites\t$unPhasedSites") >> phaseVal_${chrom}.txt
        echo -e `basename $sampleFolder` "-> Done"
        
        rm tmpSample.txt
    done
done

Chr1
Chr1_covHigh_10x-10_Am_Pla_pb3403_v3.5 -> Done
Chr1_covHigh_10x-11_Am_Pla_pb4318_v3.5 -> Done
Chr1_covHigh_10x-12_Am_Pla_pb4563_v3.5 -> Done
Chr1_covHigh_10x-1_Am_Pla_pb0174_v3.5 -> Done
Chr1_covHigh_10x2-13_Am_Pla_pb1196_v3.5 -> Done
Chr1_covHigh_10x2-14_Am_Pla_pb1008_v3.5 -> Done
Chr1_covHigh_10x2-15_Am_Pla_pb1253_v3.5 -> Done
Chr1_covHigh_10x2-16_Am_Pla_pb1682_v3.5 -> Done
Chr1_covHigh_10x2-17_Am_Pla_pb2190_v3.5 -> Done
Chr1_covHigh_10x2-18_Am_Pla_pb1687_v3.5 -> Done
Chr1_covHigh_10x-2_Am_Pla_pb0216_v3.5 -> Done
Chr1_covHigh_10x-3_Am_Pla_pb0294_v3.5 -> Done
Chr1_covHigh_10x-4_Am_Pla_pb0349_v3.5 -> Done
Chr1_covHigh_10x-5_Am_Pla_pb0403_v3.5 -> Done
Chr1_covHigh_10x-6_Am_Pla_pb0458_v3.5 -> Done
Chr1_covHigh_10x-7_Am_Pla_pb0407_v3.5 -> Done
Chr1_covHigh_10x-8_Am_Pla_pb0768_v3.5 -> Done
Chr1_covHigh_10x-9_Am_Pla_pb2333_v3.5 -> Done
Chr1_covHigh_10xNEW-Plate10-10_Am_Pla_pb0952_v3.5 -> Done
Chr1_covHigh_10xNEW-Plate10-11_Am_Pla_pb0960_v3.5 -> Done
Chr1_covHigh_10xNEW-Plate10-12_Am_Pl

## RosEl phaseblocks

### Sample1: Chr6_covHigh_60x-1_Am_Pla_pb0464_v3.5

In [ ]:
## Read input
chrom=Chr6
chromStart_buffer=52500000
chromEnd_buffer=53500000
chromStart_RosEl=52775000
chromEnd_RosEl=53150000

sample=Chr6_covHigh_60x-1_Am_Pla_pb0464_v3.5

## Define variables
baseDIR=/nfs/scistore18/bartogrp/apal/snap_hap/variants/molphase/${chrom}/indv-molphase
sampleDIR=${baseDIR}/${sample}

## Got to working directory
cd $sampleDIR
pwd

In [4]:
## gzip and index HET VCF
if [ ! -e *.het.vcf.gz ]; 
then 
    bgzip *.het.vcf; 
fi 

hetVCF=(*.het.vcf.gz)
echo hetVCF: $hetVCF

bcftools tabix -f $hetVCF

hetVCF: Chr6_covHigh_60x-1_Am_Pla_pb0464_v3.5.het.vcf.gz


In [ ]:
# Get no. of total variants
echo "Variants in RosEl+buffer:" `bcftools view -H -r $chrom:${chromStart_buffer}-${chromEnd_buffer} $hetVCF | wc -l`
echo "Variants in RosEl:" `bcftools view -H -r $chrom:${chromStart_RosEl}-${chromEnd_RosEl} $hetVCF | wc -l`

Variants in RosEl+buffer: 10897  
Variants in RosEl: 3888

In [ ]:
# Get no. of molphased variants

phaseBlockFile=(*.hapcut2.output)
echo $phaseBlockFile

echo "Phased Variants in RosEl+buffer:" \
    `awk -v chromStart_buffer=${chromStart_buffer} \
        -v chromEnd_buffer=${chromEnd_buffer} \
        '$5 > chromStart_buffer && $5 < chromEnd_buffer {count++} END {print count}' $phaseBlockFile`

echo "Phased Variants in RosEl:" \
    `awk -v chromStart_RosEl=${chromStart_RosEl} \
        -v chromEnd_RosEl=${chromEnd_RosEl} \
        '$5 > chromStart_RosEl && $5 < chromEnd_RosEl {count++} END {print count}' $phaseBlockFile`

Chr6_covHigh_60x-1_Am_Pla_pb0464_v3.5_thres10.hapcut2.output   
Phased Variants in RosEl+buffer: 10700   
Phased Variants in RosEl: 3827

In [16]:
# Print mol-phaseblocks in RosEl
phaseBlockBED=(*.hapcut2.bed)

cat $phaseBlockBED | \
    awk -v start=${chromStart_RosEl} -v end=${chromEnd_RosEl} \
        '!($3 < start) && !($2 > end)' | \
    sort -k 8nr

Chr6	45167013	55662924	Chr6:45167013-55662924:95857:10495911:618514	350632	101218	95857	10495911	618514
Chr6	53128522	53132181	Chr6:53128522-53132181:4:3659:77	427813	4	4	3659	77
Chr6	52938340	52939012	Chr6:52938340-52939012:13:672:170	425877	13	13	672	170
Chr6	52788415	52788834	Chr6:52788415-52788834:10:419:16	424359	10	10	419	16
Chr6	53043020	53043318	Chr6:53043020-53043318:3:298:4	426919	3	3	298	4
Chr6	52787198	52787472	Chr6:52787198-52787472:7:274:29	424352	7	7	274	29
Chr6	53058632	53058842	Chr6:53058632-53058842:2:210:2	427014	2	2	210	2
Chr6	53115916	53116076	Chr6:53115916-53116076:2:160:10	427684	2	2	160	10
Chr6	52876790	52876948	Chr6:52876790-52876948:3:158:77	425378	3	3	158	77
Chr6	53146728	53146853	Chr6:53146728-53146853:6:125:11	427982	6	6	125	11
Chr6	53147591	53147715	Chr6:53147591-53147715:3:124:18	427989	3	3	124	18
Chr6	52792422	52792513	Chr6:52792422-52792513:3:91:6	424392	3	3	91	6
Chr6	53056194	53056282	Chr6:53056194-53056282:4:88:1	427009	4	4	88	1
Chr6	52788978	52789061

In [17]:
## Print a particular block stats
# Chr6	53128522	53132181	Chr6:53128522-53132181:4:3659:77	427813	4	4	3659	77

phased=4
span=3659
block="SPAN: $span"

echo $block
grep -A $((phased+1)) "$block" *output | awk -v start=${chromStart_RosEl} -v end=${chromEnd_RosEl} '!($5 < start) && !($5 > end) {count++} END {print count}'
grep -A $((phased+1)) "$block" *output | awk -v start=${chromStart_buffer} -v end=${chromEnd_buffer} '!($5 < start) && !($5 > end) {count++} END {print count}'

SPAN: 3659
4
4


In [18]:
## Print a particular block stats
# Chr6	45167013	55662924	Chr6:45167013-55662924:95857:10495911:618514	350632	101218	95857	10495911	618514

phased=95857
span=10495911
block="SPAN: $span"

echo $block
grep -A $((phased+1)) "$block" *output | awk -v start=${chromStart_RosEl} -v end=${chromEnd_RosEl} '!($5 < start) && !($5 > end) {count++} END {print count}'
grep -A $((phased+1)) "$block" *output | awk -v start=${chromStart_buffer} -v end=${chromEnd_buffer} '!($5 < start) && !($5 > end) {count++} END {print count}'

SPAN: 10495911
3732
10374


### all Samples

In [ ]:
module load bcftools/1.18

## Read input
chrom=Chr6
chromStart_buffer=52500000
chromEnd_buffer=53500000
chromStart_RosEl=52775000
chromEnd_RosEl=53150000

baseDIR=/nfs/scistore18/bartogrp/apal/snap_hap/variants/molphase/${chrom}/indv-molphase
cd $baseDIR

for sample in $baseDIR/*cov*
do
    echo $sample
    cd $sample

    ## Read variables
    phaseBlockBED=(*.hapcut2.bed)
    blockFile=(*.hapcut2.output)

    ## Get phaseBlocks in RosEl region
    awk -v start=${chromStart_RosEl} -v end=${chromEnd_RosEl} '!($3 < start) && !($2 > end)' $phaseBlockBED | \
        sort -k 8nr > ${phaseBlockBED/_thres10.hapcut2.bed/_RosEl.bed}
    # if [ -e ${phaseBlockBED/_thres10.hapcut2.bed/_RosEl.tmp.bed} ]; then rm ${phaseBlockBED/_thres10.hapcut2.bed/_RosEl.tmp.bed}; fi
	
    while read chrom chromStart chromEnd blockName offset len phased span frags;
	do
		block="BLOCK: offset: $offset len: $len phased: $phased SPAN: $span fragments $frags"
		phased_RosEl=`grep -A $((phased+1)) "$block" $blockFile | awk -v start=${chromStart_RosEl} -v end=${chromEnd_RosEl} '!($5 < start) && !($5 > end) {count++} END {print count}'`
		echo -e $chrom"\t"$chromStart"\t"$chromEnd"\t"$blockName"\t"$offset"\t"$len"\t"$phased"\t"$span"\t"$frags"\t"${phased_RosEl} >> ${phaseBlockBED/_thres10.hapcut2.bed/_RosEl.tmp.bed}
	done < ${phaseBlockBED/_thres10.hapcut2.bed/_RosEl.bed}
	mv ${phaseBlockBED/_thres10.hapcut2.bed/_RosEl.tmp.bed} ${phaseBlockBED/_thres10.hapcut2.bed/_RosEl.bed}

    
    ## Get phaseBlocks in RosEl+buffer region
    awk -v start=${chromStart_buffer} -v end=${chromEnd_buffer} '!($3 < start) && !($2 > end)' $phaseBlockBED | \
    	sort -k 8nr > ${phaseBlockBED/_thres10.hapcut2.bed/_RosEl+buffer.bed}
    # if [ -e ${phaseBlockBED/_thres10.hapcut2.bed/_RosEl+buffer.tmp.bed} ]; then rm ${phaseBlockBED/_thres10.hapcut2.bed/_RosEl+buffer.tmp.bed}; fi
	while read chrom chromStart chromEnd blockName offset len phased span frags;
	do
		block="BLOCK: offset: $offset len: $len phased: $phased SPAN: $span fragments $frags"
		phased_buffer=`grep -A $((phased+1)) "$block" $blockFile | awk -v start=${chromStart_buffer} -v end=${chromEnd_buffer} '!($5 < start) && !($5 > end) {count++} END {print count}'`
		echo -e $chrom"\t"$chromStart"\t"$chromEnd"\t"$blockName"\t"$offset"\t"$len"\t"$phased"\t"$span"\t"$frags"\t"${phased_buffer} >> ${phaseBlockBED/_thres10.hapcut2.bed/_RosEl+buffer.tmp.bed}
	done < ${phaseBlockBED/_thres10.hapcut2.bed/_RosEl+buffer.bed}
	mv ${phaseBlockBED/_thres10.hapcut2.bed/_RosEl+buffer.tmp.bed} ${phaseBlockBED/_thres10.hapcut2.bed/_RosEl+buffer.bed}
done

### Molphase summary for RosEl region

In [ ]:
module load bcftools/1.18 datamash

## Read input
chrom=Chr6
chromStart_buffer=52500000
chromEnd_buffer=53500000
chromStart_RosEl=52775000
chromEnd_RosEl=53150000

baseDIR=/nfs/scistore18/bartogrp/apal/snap_hap/variants/molphase/${chrom}/indv-molphase
cd $baseDIR

summaryRosEl=/nfs/scistore18/bartogrp/apal/snap_hap/variants/molphase/${chrom}/phaseSummary_RosEl.txt
if [ -e $summaryRosEl]; then rm $summaryRosEl; fi
echo -e sample"\t"SNPs_RosEl"\t"phased_RosEl"\t"phased_RosEl_Top5"\t"phased_RosEl_Top3"\t"SNPs_buffer"\t"phased_buffer"\t"phased_buffer_Top5"\t"phased_buffer_Top3 > $summaryRosEl

for sample in $baseDIR/*
do
    echo $sample
    cd $sample
    
	## gzip and index HET VCF
	# if [ ! -e *.het.vcf.gz ]; then bgzip *.het.vcf; fi 
	bgzip -f *het.vcf
    hetVCF=(*.het.vcf.gz)
	echo hetVCF: $hetVCF
	bcftools tabix -f $hetVCF

	## Count SNPs in RosEl 
	SNPs_RosEl=`bcftools view -H -r $chrom:${chromStart_RosEl}-${chromEnd_RosEl} $hetVCF | wc -l`
    ## Count phased SNPs in RosEl 
    blocks_RosEl=(*RosEl.bed)
    phased_RosEl=`sort -k 10nr $blocks_RosEl | datamash sum 10`
    ## Count phased SNPs in RosEl that are covered by the longest phaseBlocks 
    phased_RosEl_Top1=`sort -k 10nr $blocks_RosEl | head -1 | datamash sum 10`
    phased_RosEl_Top3=`sort -k 10nr $blocks_RosEl | head -3 | datamash sum 10`
    phased_RosEl_Top5=`sort -k 10nr $blocks_RosEl | head -5 | datamash sum 10`

    
	## Count SNPs in RosEl+buffer 
	SNPs_buffer=`bcftools view -H -r $chrom:${chromStart_buffer}-${chromEnd_buffer} $hetVCF | wc -l`
	## Count phased SNPs in RosEl+buffer
    blocks_buffer=(*RosEl+buffer.bed)
    phased_buffer=`sort -k 10nr $blocks_buffer | datamash sum 10`
	## Count phased SNPs in RosEl+buffer that are covered by the largest phaseBlocks
    phased_buffer_Top1=`sort -k 10nr $blocks_buffer | head -1 | datamash sum 10`
    phased_buffer_Top3=`sort -k 10nr $blocks_buffer | head -3 | datamash sum 10`
    phased_buffer_Top5=`sort -k 10nr $blocks_buffer | head -5 | datamash sum 10`

    echo -e ${sample}"\t"${SNPs_RosEl}"\t"${phased_RosEl}"\t"${phased_RosEl_Top5}"\t"${phased_RosEl_Top3}"\t"${phased_RosEl_Top1}"\t"${SNPs_buffer}"\t"${phased_buffer}"\t"${phased_buffer_Top5}"\t"${phased_buffer_Top3}"\t"${phased_buffer_Top1} >> $summaryRosEl
done